In [ ]:
import pandas as pd
import requests
import os
import time

GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY_HERE"

def get_lat_long(address):
    """
    Calls Google Geocoding API and returns (lat, lng)
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": GOOGLE_API_KEY}

    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]

    return None, None

def process_province_xls(input_file, province_code="AB"):
    df = pd.read_excel(input_file)

    df = df[df["Site Province Abbrev"] == province_code]

    df = df.rename(columns={
        "Establishment Name": "StoreName",
        "Site City Name": "City",
        "Site Province Abbrev": "Province",
        "Site Address Line 1": "Address",
        "Site Postal Code": "PostalCode"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng = get_lat_long(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng

        time.sleep(0.2)

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


def append_to_master_csv(df, output_file=r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\02_processed_data\01_store_locations\Cannabis_Store_Locations.csv"):
    if not os.path.exists(output_file):
        df.to_csv(output_file, index=False)
    else:
        df.to_csv(output_file, index=False, mode='a', header=False)

    print("Saved/Updated:", output_file)


# Run the pipeline
df_cleaned = process_province_xls(r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls")
append_to_master_csv(df_cleaned)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 9827 279 ST, ACHESON, Alberta, Canada
Fetching coordinates for: 1005-401 COOPERS BLVD SW, AIRDRIE, Alberta, Canada
Fetching coordinates for: 4A-1861 MEADOWBROOK DR SE, AIRDRIE, Alberta, Canada
Fetching coordinates for: 17-1301 8 STREET SW, AIRDRIE, Alberta, Canada
Fetching coordinates for: 600-705 MAIN ST SW, AIRDRIE, Alberta, Canada
Fetching coordinates for: 101-400 MAIN ST NE, AIRDRIE, Alberta, Canada
Fetching coordinates for: 6128-403 MACKENZIE WAY SW, AIRDRIE, Alberta, Canada
Fetching coordinates for: 117-1800 MARKET ST SE, AIRDRIE, Alberta, Canada
Fetching coordinates for: 5-213 MAIN ST N, AIRDRIE, Alberta, Canada
Fetching coordinates for: 130-2765 MAIN ST SW, AIRDRIE, Alberta, Canada
Fetching coordinates for: 4925 50 AVENUE, ALBERTA BEACH, Alberta, Canada
Fetching coordinates for: 450153 82 STREET E, ALDERSYDE, Alberta, Canada
Fetching coordinates for: 103-5003 50 AVENUE, ATHABASCA, A

In [25]:
# Run for British Columbia
df_bc = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="BC"
)

append_to_master_csv(df_bc)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 31270 WHEEL AVE, ABBOTSFORD, British Columbia, Canada
Fetching coordinates for: 27756 NATHAN PLACE, ABBOTSFORD, British Columbia, Canada
Fetching coordinates for: 30523 BURGESS AVENUE, ABBOTSFORD, British Columbia, Canada
Fetching coordinates for: 5728 LOUGHEED HIGHWAY, AGASSIZ, British Columbia, Canada
Fetching coordinates for: 2-3107 HENRY RD, CHEMAINUS, British Columbia, Canada
Fetching coordinates for: 3316 ROBSON DRIVE, COQUITLAM, British Columbia, Canada
Fetching coordinates for: 3-68 SCHOONER STREET, COQUITLAM, British Columbia, Canada
Fetching coordinates for: 3391 CRESTON CLOSE, COQUITLAM, British Columbia, Canada
Fetching coordinates for: 900 INDUSTRIAL ROAD 1, CRANBROOK, British Columbia, Canada
Fetching coordinates for: 896 RECLAMATION ROAD, CRESTON, British Columbia, Canada
Fetching coordinates for: 4431 80 STREET, DELTA, British Columbia, Canada
Fetching coordinates for: 3695 

In [ ]:
# Run for British Columbia CSV with different format
import re

def clean_full_address(full_address, city, province):
    """
    Removes city + province from the end of the FullAddress.
    Extracts postal code.
    """

    postal_code_pattern = r"[A-Za-z]\d[A-Za-z]\s?\d[A-Za-z]\d"
    postal_match = re.search(postal_code_pattern, full_address)

    postal_code = postal_match.group(0) if postal_match else None

    cleaned = full_address

    remove_pattern = rf",\s*{re.escape(city)}\s*,\s*{province}\s*{postal_code}"
    cleaned = re.sub(remove_pattern, "", cleaned)

    cleaned = cleaned.strip()

    return cleaned, postal_code


def process_bc_file(input_file):
    df = pd.read_csv(input_file)

    df["Address"] = None
    df["PostalCode"] = None
    df["Latitude"] = None
    df["Longitude"] = None

    for idx, row in df.iterrows():
        full_addr = row["FullAddress"]
        city = row["City"]
        province = row["Province"]

        addr_clean, postal_code = clean_full_address(full_addr, city, province)

        df.at[idx, "Address"] = addr_clean
        df.at[idx, "PostalCode"] = postal_code

        geo_addr = f"{addr_clean}, {city}, {row['FullProvinceName']}, Canada"

        lat, lng = get_lat_long(geo_addr)
        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng

        time.sleep(0.2)

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df

df_bc_cleaned = process_bc_file(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\BritishColumbia.csv"
)

append_to_master_csv(df_bc_cleaned)

Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv


In [ ]:
# Run for Manitoba
df_mb = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="MB"
)

append_to_master_csv(df_mb)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 88-135 HIGHWAY 353, BROOKDALE, Manitoba, Canada
Fetching coordinates for: 760 PANDORA AVE E, WINNIPEG, Manitoba, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv


In [ ]:
# Run for Manitoba CSV with different format
def get_lat_long_postal(address):
    """
    Calls Google Geocoding API and returns (lat, lng, postal_code)
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": GOOGLE_API_KEY}

    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] == "OK":
        result = data["results"][0]
        location = result["geometry"]["location"]
        lat = location["lat"]
        lng = location["lng"]

        # Extract postal code
        postal_code = None
        for component in result["address_components"]:
            if "postal_code" in component["types"]:
                postal_code = component["long_name"]
                break

        return lat, lng, postal_code

    return None, None, None


def process_manitoba_province_csv(input_file, province_code="MB"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_mb = process_manitoba_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Manitoba.csv",
    province_code="MB"
)

append_to_master_csv(df_mb)

Fetching coordinates for: 61 Centre Avenue East, Altona, Manitoba, Canada
Fetching coordinates for: Unit 1 - 45 4th Avenue Northeast, Altona, Manitoba, Canada
Fetching coordinates for: 6163 PTH 30, Altona, Manitoba, Canada
Fetching coordinates for: 108 Kinosota Road South, Amaranth, Manitoba, Canada
Fetching coordinates for: 934 Dugald Road, Anola, Manitoba, Canada
Fetching coordinates for: 145 Sunset Boulevard, Arborg, Manitoba, Canada
Fetching coordinates for: 20 Main Street, Ashern, Manitoba, Canada
Fetching coordinates for: 724 Park Avenue, Beausejour, Manitoba, Canada
Fetching coordinates for: 612 Park Avenue, Beausejour, Manitoba, Canada
Fetching coordinates for: Unit 1 - 550 Mill Road, Boissevain, Manitoba, Canada
Fetching coordinates for: Unit B - 1860 18th Street North, Brandon, Manitoba, Canada
Fetching coordinates for: Unit 4 - 930 18th Street, Brandon, Manitoba, Canada
Fetching coordinates for: Unit 48 - 1570 18th Street, Brandon, Manitoba, Canada
Fetching coordinates for: 

In [33]:
# Run for New Brunswick
df_nb = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="NB"
)

append_to_master_csv(df_nb)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 35 ENGLISH DRIVE, MONCTON, New Brunswick, Canada
Fetching coordinates for: 300-814 MAIN STREET, MONCTON, New Brunswick, Canada
Fetching coordinates for: 9-78 MILLTOWN BOULEVARD, ST STEPHEN, New Brunswick, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv


In [ ]:
# Run for New Brunswick CSV with different format

def process_new_brunswick_province_csv(input_file, province_code="NB"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng = get_lat_long(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_nb = process_new_brunswick_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\NewBrunswick.csv",
    province_code="NB"
)

append_to_master_csv(df_nb)

Fetching coordinates for: 640 ave St. Peter Ave, Suite #PD201, Bathurst, New Brunswick, Canada
Fetching coordinates for: 157 rue Water Street, Unit 14, Campbellton, New Brunswick, Canada
Fetching coordinates for: 784 boul Dieppe Blvd, Dieppe, New Brunswick, Canada
Fetching coordinates for: 575B rue Victoria Street, Edmundston, New Brunswick, Canada
Fetching coordinates for: 435 prom. Brookside Drive, Unit A002, Fredericton, New Brunswick, Canada
Fetching coordinates for: 334 Queen St, Fredericton, New Brunswick, Canada
Fetching coordinates for: 1735 Hanwell Rd unit 3, Hanwell, New Brunswick, Canada
Fetching coordinates for: 45 allée Woodside Lane, Fredericton, New Brunswick, Canada
Fetching coordinates for: 180 ch Madawaska Road, Unit #165, Grand-Falls, New Brunswick, Canada
Fetching coordinates for: 2530 route King George Highway, Miramichi, New Brunswick, Canada
Fetching coordinates for: 50 prom. Granite Drive, Unit 102, Moncton, New Brunswick, Canada
Fetching coordinates for: 165 ru

In [ ]:
# Run for New Foundland and Labrador
df_nl = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="NL"
)

append_to_master_csv(df_nl)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 1 SEAVIEW DRIVE, BURIN, Newfoundland and Labrador, Canada
Fetching coordinates for: 50 CAPTAIN PRIM DR, ST. JOHN'S, Newfoundland and Labrador, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv


In [ ]:
# Run for New Foundland and Labrador CSV with different format

def process_newfoundland_province_csv(input_file, province_code="NL"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_nl = process_newfoundland_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Newfoundland.csv",
    province_code="NL"
)

append_to_master_csv(df_nl)

Fetching coordinates for: 27 Aspen Drive, Happy Valley-Goose Bay, Newfoundland and Labrador, Canada
Fetching coordinates for: 230 Airport Boulevard, Gander, Newfoundland and Labrador, Canada
Fetching coordinates for: 100 Laurell Road, Gander, Newfoundland and Labrador, Canada
Fetching coordinates for: 100 Canada Drive, Harbour Brenton, Newfoundland and Labrador, Canada
Fetching coordinates for: 56 Main Street, Glovertown, Newfoundland and Labrador, Canada
Fetching coordinates for: 242 Memorial Drive, Clarenville, Newfoundland and Labrador, Canada
Fetching coordinates for: 47 Main Street, Burin, Newfoundland and Labrador, Canada
Fetching coordinates for: 132 Trans Canada Hwy, Clarenville, Newfoundland and Labrador, Canada
Fetching coordinates for: 6 Sweetland's Hill, Bonavista, Newfoundland and Labrador, Canada
Fetching coordinates for: 71 Blockhouse Rd., Placentia, Newfoundland and Labrador, Canada
Fetching coordinates for: 667 A Trans Canada Highway, Whitbourne, Newfoundland and Labra

In [ ]:
# Run for Northwest Territories
def process_northwest_territories_province_csv(input_file, province_code="NT"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_nt = process_northwest_territories_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\NorthwestTerritories.csv",
    province_code="NT"
)

append_to_master_csv(df_nt)

Fetching coordinates for: 1-4 Courtoreille Street, Hay River, Northwest Territories, Canada
Fetching coordinates for: 110 Veteran’s Way, Inuvik, Northwest Territories, Canada
Fetching coordinates for: 5 Town Square, Norman Wells, Northwest Territories, Canada
Fetching coordinates for: 5123 51 St (Kingpin Centre Building), Yellowknife, Northwest Territories, Canada
Fetching coordinates for: Unit 12, 100 Borden Drive, Yellowknife, Northwest Territories, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv


In [ ]:
# Run for Nova Scotia
df_ns = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="NS"
)

append_to_master_csv(df_ns)

# Run for Nova Scotia CSV with different format
def process_nova_scotia_province_csv(input_file, province_code="NS"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_ns_new = process_nova_scotia_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\NovaScotia.csv",
    province_code="NS"
)

append_to_master_csv(df_ns_new)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 31 ORION CRT, DARTMOUTH, Nova Scotia, Canada
Fetching coordinates for: 41 ESTATES ROAD, LOWER SACKVILLE, Nova Scotia, Canada
Fetching coordinates for: 115 HARTIGAN DRIVE, SYDNEY MINES, Nova Scotia, Canada
Fetching coordinates for: 485 INDUSTRIAL AVENUE, TRURO, Nova Scotia, Canada
Fetching coordinates for: 15693 NS-4, WENTWORTH, Nova Scotia, Canada
Fetching coordinates for: 50 IVEY LANE, WINDSOR, Nova Scotia, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv
Fetching coordinates for: 126 South Albion Street, Amherst, Nova Scotia, Canada
Fetching coordinates for: 180 St. Anthony Street, Annapolis Royal, Nova Scotia, Canada
Fetching coordinates for: 151 Church Street, Antigonish, Nova Scotia, Canada
Fetching coordinates for: 16 Jessica Wong Lane,

In [ ]:
# Run for Nunavut CSV with different format
def process_nunavut_province_csv(input_file, province_code="NU"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_nu = process_nunavut_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Nunavut.csv",
    province_code="NU"
)

append_to_master_csv(df_nu)

Fetching coordinates for: 1501 Federal Road, Iqaluit, Nunavut, Canada
Fetching coordinates for: PO Box 304, Rankin Inlet, Nunavut, Canada
Fetching coordinates for: 760 Queen Elizabeth Way, Iqaluit, Nunavut, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv


In [ ]:
# Run for Ontario
df_on = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="ON"
)

append_to_master_csv(df_on)

# Run for Ontario CSV with different format
def process_ontario_province_csv(input_file, province_code="ON"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "Store Name": "StoreName",
        "Municipality or First Nation": "City",
    })
    df["Province"] = province_code

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)
    
    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.3)  

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_on_new = process_ontario_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Ontario.csv",
    province_code="ON"
)

append_to_master_csv(df_on_new)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 250 ELM STREET, AYLMER, Ontario, Canada
Fetching coordinates for: 151 JOHN STREET, BARRIE, Ontario, Canada
Fetching coordinates for: 31 HANSEN ROAD SOUTH, BRAMPTON, Ontario, Canada
Fetching coordinates for: 2-47 MORTON AVENUE EAST, BRANTFORD, Ontario, Canada
Fetching coordinates for: 11 BODINE DR, BRANTFORD, Ontario, Canada
Fetching coordinates for: 104-566 RIVERVIEW DRIVE, CHATHAM, Ontario, Canada
Fetching coordinates for: 2741-42 COUNTRY ROAD, CLEARVIEW, Ontario, Canada
Fetching coordinates for: 18 CANSO RD, ETOBICOKE, Ontario, Canada
Fetching coordinates for: 452A VALERMO DR, ETOBICOKE, Ontario, Canada
Fetching coordinates for: 5-130 NORTH QUEEN STREET, ETOBICOKE, Ontario, Canada
Fetching coordinates for: 333 JARVIS STREET, FORT ERIE, Ontario, Canada
Fetching coordinates for: 2420 HIGHWAY 6, JARVIS, Ontario, Canada
Fetching coordinates for: 1915 JERSEYVILLE RD W, JERSEYVILLE, Ontario, Ca

In [ ]:
# Run for Prince Edward Island
df_pe = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="PE"
)

append_to_master_csv(df_pe)

# Run for Prince Edward Island CSV with different format

def process_prince_edward_island_province_csv(input_file, province_code="PE"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng = get_lat_long(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng

        time.sleep(0.2) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_pe_new = process_prince_edward_island_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\PrinceEdwardIsland.csv",
    province_code="PE"
)

append_to_master_csv(df_pe_new)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 11 REGIS DUFFY DRIVE, CHARLOTTETOWN, Prince Edward Island, Canada
Fetching coordinates for: 7 INNOVATION WAY, CHARLOTTETOWN, Prince Edward Island, Canada
Fetching coordinates for: 200-25 FOURTH ST, CHARLOTTETOWN, Prince Edward Island, Canada
Fetching coordinates for: 2-16 MYRTLE STREET, STRATFORD, Prince Edward Island, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv
Fetching coordinates for: 85 Belvedere Ave, Charlottetown, Prince Edward Island, Canada
Fetching coordinates for: 509 Main St, PO Box 1601, Montague, Prince Edward Island, Canada
Fetching coordinates for: 478 Main St, O’Leary, Prince Edward Island, Canada
Fetching coordinates for: 447 Granville St, Summerside, Prince Edward Island, Canada
Fetching coordinates for: 9 Kinlock Road,

In [ ]:
# Run for Quebec
df_qc = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="QC"
)

append_to_master_csv(df_qc)

# Run for Quebec CSV with different format
def process_Quebec_province_csv(input_file, province_code="QC"):
    df = pd.read_csv(input_file)


    df = df.rename(columns={
        "FullAddress": "Address"
    })


    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)


    df["Latitude"] = None
    df["Longitude"] = None


    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng = get_lat_long(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng

        time.sleep(0.2) 


    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_qc_new = process_Quebec_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Quebec.csv",
    province_code="QC"
)

append_to_master_csv(df_qc_new)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: 101-4170 BOULEVARD LA PRADE, BÉCANCOUR, Quebec, Canada
Fetching coordinates for: 6605 RUE YVON-TRUDEAU, BÉCANCOUR, Quebec, Canada
Fetching coordinates for: 84 RUE DE LA GARE RD, CARLETON, Quebec, Canada
Fetching coordinates for: 1144 MAGENTA BOULEVARD ESTATES, FARNHAM, Quebec, Canada
Fetching coordinates for: 2295 CHEMIN RIDGE, HUNTINGDON, Quebec, Canada
Fetching coordinates for: 11 BEACON ROAD, KIRKLAND, Quebec, Canada
Fetching coordinates for: 7625 ROUTE ARTHUR SAUVE, MIRABEL, Quebec, Canada
Fetching coordinates for: 13065 RTE ARTHUR-SAUVÉ, MIRABEL, Quebec, Canada
Fetching coordinates for: 1010 SHERBROOKE OUEST SUITE 1800, MONTRÉAL, Quebec, Canada
Fetching coordinates for: 216 CHENAL TARDIF, PIERREVILLE, Quebec, Canada
Fetching coordinates for: 243 BOULEVARD HYMUS, POINTE CLAIRE, Quebec, Canada
Fetching coordinates for: 815B-C AVENUE TECUMSEH, POINTE CLAIRE, Quebec, Canada
Fetching coordi

In [ ]:
# Run for Saskatchewan
df_sk = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="SK"
)

append_to_master_csv(df_sk)

# Run for Saskatchewan CSV with different format
def process_saskatchewan_province_csv(input_file, province_code="SK"):
    df = pd.read_csv(input_file)
    
    df["Operating Name"] = df["Operating Name"].astype(str)
    df["Operating Name"] = df["Operating Name"].str.split(" - ").str[0].str.strip()
   
    df = df.rename(columns={
        "Street Address": "Address",
        "Operating Name": "StoreName",
    })
    df["Province"] = province_code

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None
    df["PostalCode"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng, postal = get_lat_long_postal(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng
        df.at[idx, "PostalCode"] = postal

        time.sleep(0.3) 

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_sk_new = process_saskatchewan_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Saskatchewan.csv",
    province_code="SK"
)

append_to_master_csv(df_sk_new)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Fetching coordinates for: PARCEL A SE QUARTER SECTION 14 TOWNSHIP 25 R4 W3, CORMAN PARK NO.344, Saskatchewan, Canada
Fetching coordinates for: 3434 RIDEOUT BAY, REGINA, Saskatchewan, Canada
Fetching coordinates for: 3476 SASKATCHEWAN DR, REGINA, Saskatchewan, Canada
Fetching coordinates for: 2303 EMMET HALL ROAD, REGINA, Saskatchewan, Canada
Fetching coordinates for: 219 APEX ST, SASKATOON, Saskatchewan, Canada
Fetching coordinates for: 306234 TWP ROAD 380, SASKATOON, Saskatchewan, Canada
Fetching coordinates for: 1902 ALBERTA AVENUE, SASKATOON, Saskatchewan, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv
Fetching coordinates for: 1015 Main St., Kindersley, Saskatchewan, Canada
Fetching coordinates for: 116 Centre St, Regina Beach, Saskatchewan, Canada
Fetching coor

In [ ]:
# Run for Yukon
df_yt = process_province_xls(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Alberta.xls",
    province_code="YT"
)

append_to_master_csv(df_yt)

# Run for Yukon CSV with different format
def process_yukon_province_csv(input_file, province_code="YT"):
    df = pd.read_csv(input_file)

    df = df.rename(columns={
        "FullAddress": "Address"
    })

    PROVINCE_FULL = {
        "AB": "Alberta", "BC": "British Columbia", "SK": "Saskatchewan",
        "MB": "Manitoba", "ON": "Ontario", "QC": "Quebec",
        "NB": "New Brunswick", "NS": "Nova Scotia", "PE": "Prince Edward Island",
        "NL": "Newfoundland and Labrador", "YT": "Yukon",
        "NT": "Northwest Territories", "NU": "Nunavut"
    }
    df["FullProvinceName"] = df["Province"].map(PROVINCE_FULL)

    df["Latitude"] = None
    df["Longitude"] = None

    df["FullAddress"] = (
        df["Address"] + ", " +
        df["City"] + ", " +
        df["FullProvinceName"] + ", Canada"
    )

    for idx, row in df.iterrows():
        print(f"Fetching coordinates for: {row['FullAddress']}")

        lat, lng = get_lat_long(row["FullAddress"])

        df.at[idx, "Latitude"] = lat
        df.at[idx, "Longitude"] = lng

        time.sleep(0.2)  

    df = df[[
        "StoreName",
        "City",
        "Province",
        "FullProvinceName",
        "Address",
        "PostalCode",
        "Latitude",
        "Longitude"
    ]]

    return df


df_yt_new = process_yukon_province_csv(
    r"C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Yukon.csv",
    province_code="YT"
)

append_to_master_csv(df_yt_new)

WARNING *** file size (355189) not 512 + multiple of sector size (512)
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv
Fetching coordinates for: 978 Second Ave, Dawson City, Yukon, Canada
Fetching coordinates for: 80 Chillkot Way, Whitehorse, Yukon, Canada
Fetching coordinates for: 204C Main St, Whitehorse, Yukon, Canada
Fetching coordinates for: 120b Industrial Rd, Whitehorse, Yukon, Canada
Fetching coordinates for: 211 Wood St, Whitehorse, Yukon, Canada
Fetching coordinates for: 516 Adela Trail, Watson Lake, Yukon, Canada
Saved/Updated: C:\Users\aayus\Desktop\McMaster\Term 1\Data analytics and big data\Project\SEP_6DA3_Retail_Access_vs_Social_Cost\data\01_raw_data\01_store_locations\Cannabis_Store_Locations.csv
